# MD-SAPT: Python Based Toolkit for Running Symmetry Adapted Perturbation Theory Calculations of Molecular Dynamics Trajectories
### Alia Lescoulie, Astrid Yu, and Ashley Ringer McDonald

This Jupyter notebook demonstrates the MD-SAPT interactive user experience, showing the whole process for an example polypeptide from input to analysis.

In [1]:
import mdsapt

/home/alia/.conda/envs/mdsapt/lib/python3.9/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")


2022-01-20 12:49:57,791 mdsapt       INFO     MDSAPT full-release starting
2022-01-20 12:49:57,792 mdsapt       INFO     Copyright (c) 2021 Alia Lescoulie, Astrid Yu, and Ashley Ringer McDonald
2022-01-20 12:49:57,793 mdsapt       INFO     Released under MIT Licence


## Step 1: Input file

MD-SAPT uses YAML a type of plain text file to pass input setting into the code. Below is a copy of the input file uses in this demo. The input file contains all the information needed to run the SAPT calculations on the trajectory including: paths to the MD data, the residues of interest, the frames to run, and the computing recourses requested. Below is the input used in this notebook. Blank inputs are generated using the `mdsapt-get-input.py` script as shown in the code block.

```bash
python mdsapt-get-input.py input.yaml
```

Afterwards the input file is filled out with the needed information.

```YAML
topology_path: 'testtop.psf'
trajectory_paths:
  - 'testtraj.dcd'
selection_resid_num:
  - 11
  - 199
int_pairs:
  # Place pair of  selections defined above in a list of lists
  - [11, 199]
trajectory_settings:
  start: 78
  stop: 98
  step: 1
system_settings:
  ncpus: '12'
  memory: '12GB'
  time: '24:00:00'
opt_settings:
  pH: 7
sapt_settings:
  method: 'sapt0'
  basis: 'jun-cc-pvdz'
  settings:
    reference: 'rhf'
  save_psi4_output: true
```

In python the input file is given to the **InputReader** which checks its validity and stores the information as variables.

In [4]:
In = mdsapt.InputReader('input.yaml')

2022-01-20 12:53:24,385 mdsapt.reader INFO     Input Parameters Accepted


**InputReader** also checks the input file for errors and give the location of issues in the input file. For example if the stop frame selected in the input file exceeds the trajectory length MD-SAPT notifies you with an error message before starting the SAPT calculation and wasting cluster time.

```YAML
trajectory_settings:
  start: 78
  stop: 120
  step: 1trajectory_settings:
```

In [6]:
In2 = mdsapt.InputReader('error_input.yaml')

2022-01-20 12:59:38,860 mdsapt.reader CRITICAL Stop exceeds length of trajectory.


InputError: 

## Step 2: Preparing Residues

Residues when pulled initially from a trajectory are not able to immediatly be thrown into SAPT calcuations. They have missing protons and unpaired electrons resulting splitting the bonds forming the protein backbone. This is fixed by the **Optimizer** object in MD-SAPT. **Optimizer**, when initialized is passed **InputReader** which give it the user information.

In [7]:
Opt = mdsapt.Optimizer(In)

Residues undergo two steps in optimization, replacing amino protons, and adding a new bond on the C terminus. MD-SAPT's **Viewer** can give a continent before and after.

In [11]:
View = mdsapt.Viewer(In)

# Residue before optimization
View.view_residue(11)

NGLWidget(max_frame=97)

In [16]:
# Residue after optimization
View.view_optimized_residue(11)

/home/alia/.conda/envs/mdsapt/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:747: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/alia/.conda/envs/mdsapt/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/alia/.conda/envs/mdsapt/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/alia/.conda/envs/mdsapt/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/home/alia/.conda/envs/mdsapt/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:

NGLWidget()

Viewer also enables users to visualize the whole polypeptide as well as the interaction pairs specified in the input file.

In [13]:
View.view_system()

NGLWidget(max_frame=97)

In [14]:
View.view_optimized_interaction_pair(11, 199)

/home/alia/.conda/envs/mdsapt/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:747: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/alia/.conda/envs/mdsapt/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/alia/.conda/envs/mdsapt/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/alia/.conda/envs/mdsapt/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/home/alia/.conda/envs/mdsapt/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:

NGLWidget()